Use file 'ar41_data.csv'.
Put the notebook and data file in one folder.
Run the entire notebook

In [2]:
# from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium
from scipy import stats

In [2]:
#drive.mount('/content/drive')
#path = '/content/drive/My Drive/BDMA/Semester 1 - ULB/Personal Work/Data Mining Project/ar41_data.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
dtypes = {
    'mapped_veh_id': 'int32',
    'RS_E_InAirTemp_PC1': 'float32',
    'RS_E_InAirTemp_PC2': 'float32',
    'RS_E_OilPress_PC1': 'float32',
    'RS_E_OilPress_PC2': 'float32',
    'RS_E_RPM_PC1': 'float32',
    'RS_E_RPM_PC2': 'float32',
    'RS_E_WatTemp_PC1': 'float32',
    'RS_E_WatTemp_PC2': 'float32',
    'RS_T_OilTemp_PC1': 'float32',
    'RS_T_OilTemp_PC2': 'float32',
    'temp':'float32',
    'elevation':'float32'
}

data = pd.read_csv('ar41_data.csv', delimiter=',', dtype=dtypes)
data

,Unnamed: 0,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,...,RS_E_WatTemp_PC1,RS_E_WatTemp_PC2,RS_T_OilTemp_PC1,RS_T_OilTemp_PC2,temp,weather,weather_description,elevation,timestamps_Belgian,time_diff_overall
0,5493376,102,2023-01-23 07:25:08+00:00,51.017864,3.769079,17.0,18.0,210.0,210.0,858.0,...,78.0,80.0,71.0,79.0,1.380000,clouds,overcast clouds,13.0,2023-01-23 08:25:08+00:00,0.0
1,8969009,102,2023-01-23 07:25:16+00:00,51.017875,3.769046,17.0,20.0,200.0,200.0,801.0,...,79.0,80.0,76.0,79.0,1.380000,clouds,overcast clouds,13.0,2023-01-23 08:25:16+00:00,8.0
2,13873566,102,2023-01-23 07:25:37+00:00,51.017208,3.770179,19.0,20.0,193.0,207.0,803.0,...,80.0,81.0,79.0,81.0,1.380000,clouds,overcast clouds,12.0,2023-01-23 08:25:37+00:00,21.0
3,14994675,102,2023-01-23 07:25:41+00:00,51.016916,3.771036,19.0,20.0,196.0,203.0,801.0,...,80.0,81.0,79.0,81.0,1.380000,clouds,overcast clouds,12.0,2023-01-23 08:25:41+00:00,4.0
4,11935795,102,2023-01-23 07:26:10+00:00,51.016503,3.772182,19.0,21.0,200.0,203.0,795.0,...,80.0,82.0,79.0,79.0,1.380000,clouds,overcast clouds,13.0,2023-01-23 08:26:10+00:00,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17679234,1820536,197,2023-09-13 17:33:03+00:00,50.402693,4.450111,37.0,40.0,220.0,258.0,803.0,...,81.0,79.0,77.0,81.0,18.950001,clouds,few clouds,104.0,2023-09-13 19:33:03+00:00,5.0
17679235,17128255,197,2023-09-13 17:33:58+00:00,50.401657,4.452693,37.0,38.0,224.0,307.0,843.0,...,80.0,78.0,77.0,80.0,18.950001,clouds,few clouds,105.0,2023-09-13 19:33:58+00:00,55.0
17679236,15456252,197,2023-09-13 17:34:03+00:00,50.401830,4.452217,37.0,38.0,224.0,307.0,841.0,...,80.0,78.0,77.0,80.0,18.950001,clouds,few clouds,105.0,2023-09-13 19:34:03+00:00,5.0
17679237,7666957,197,2023-09-13 17:34:58+00:00,50.401057,4.455388,36.0,38.0,207.0,244.0,800.0,...,81.0,80.0,77.0,82.0,18.950001,clouds,few clouds,99.0,2023-09-13 19:34:58+00:00,55.0


In [7]:
data.drop(['Unnamed: 0', 'timestamps_UTC', 'weather',	'weather_description',	'elevation'], axis=1, inplace=True)

In [8]:
def calculate_speed(df, lat_col='lat', lon_col='lon', time_diff_col='time_diff_overall', veh_id_col='mapped_veh_id', journey_id_col='journey_id'):
    def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
        if to_radians:
            lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

        a = np.sin((lat2 - lat1) / 2.0)**2 + \
            np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2

        return earth_radius * 2 * np.arcsin(np.sqrt(a))
    # Calculate the speed of a train in km/h between successive rows, grouped by train and journey_id.
    def calculate_group_speed(group):
        # Calculate the distance for each group
        group['distance'] = haversine(group[lat_col].shift(), group[lon_col].shift(), group[lat_col], group[lon_col])
        # Calculate speed in km/h, avoid division by zero
        group['speed'] = np.where(group[time_diff_col] > 0, (group['distance'] / group[time_diff_col]) * 3600, np.nan)
        # Handle the first entry of each group
        group['speed'].iloc[0] = 0
        group['distance'].iloc[0] = 0
        return group
    # Group by vehicle id and journey id, then apply the speed calculation
    df = df.groupby([veh_id_col], as_index=False).apply(calculate_group_speed)
    df = df.reset_index(drop=True)
    return df

def divide_journeys(data, time_threshold_1=300, time_threshold_2=600, speed_threshold = 0.5):
    condition = (data['time_diff_overall'] >= time_threshold_1) & \
                (data['speed'] < speed_threshold) | \
                (data['time_diff_overall'] >= time_threshold_2)
    data['time_diff'] = data['time_diff_overall'].where(~condition, 0).astype(int)
    data['new_journey'] = (data['time_diff'] == 0).astype(int)
    data['journey_id'] = data.groupby('mapped_veh_id')['new_journey'].cumsum()
    data.drop('new_journey', axis=1, inplace=True)

data = calculate_speed(data)
divide_journeys(data)

In [11]:
data.to_csv('journeys.csv', index=False)

In [12]:
data[data['mapped_veh_id'] == 162]

,mapped_veh_id,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,RS_E_WatTemp_PC1,...,RS_T_OilTemp_PC1,RS_T_OilTemp_PC2,temp,timestamps_Belgian,time_diff_overall,distance,speed,time_diff,journey_id,rolling_avg_speed
10658440,162,51.013253,3.780161,37.0,15.0,13.0,3.0,0.0,0.0,37.0,...,35.0,15.0,5.55,2023-01-16 14:47:32+00:00,0.0,0.000000,0.000000,0,1,0.133253
10658441,162,51.013288,3.780150,37.0,15.0,13.0,3.0,0.0,0.0,37.0,...,36.0,14.0,5.55,2023-01-16 14:48:25+00:00,53.0,0.003924,0.266505,53,1,0.133253
10658442,162,51.013217,3.780243,37.0,15.0,13.0,3.0,0.0,0.0,37.0,...,35.0,15.0,5.55,2023-01-16 14:49:25+00:00,60.0,0.010212,0.612729,60,1,0.439617
10658443,162,51.012982,3.780711,37.0,15.0,13.0,3.0,0.0,0.0,37.0,...,36.0,15.0,5.55,2023-01-16 14:49:46+00:00,21.0,0.041873,7.178185,21,1,3.895457
10658444,162,51.013151,3.780190,37.0,15.0,13.0,3.0,0.0,0.0,37.0,...,36.0,14.0,5.55,2023-01-16 14:50:25+00:00,39.0,0.040970,3.781830,39,1,5.480008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10839287,162,50.854685,3.605392,33.0,40.0,203.0,231.0,794.0,795.0,78.0,...,82.0,89.0,14.47,2023-09-13 23:42:59+00:00,60.0,0.361663,21.699794,60,305,13.044356
10839288,162,50.855370,3.606297,32.5,40.0,150.0,184.5,488.5,553.0,77.0,...,81.5,89.0,14.47,2023-09-13 23:43:58+00:00,59.0,0.099173,6.051261,59,305,13.875528
10839289,162,50.855366,3.606301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,82.0,87.5,14.47,2023-09-13 23:44:09+00:00,11.0,0.000571,0.186945,11,305,3.119103
10839290,162,50.855359,3.606302,37.0,45.0,3.0,6.0,0.0,0.0,77.0,...,82.0,88.0,14.47,2023-09-13 23:44:29+00:00,20.0,0.000794,0.142914,20,305,0.164929


In [ ]:
data.to_csv('train162.csv', index=False)

In [9]:
columns_to_check = ['RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2']
data.dropna(subset=columns_to_check, inplace=True)

In [10]:
def rolling_operation(x):
    return x.rolling(window=2).mean().bfill().ffill()
data['rolling_avg_speed'] = data.groupby(['mapped_veh_id', 'journey_id'])['speed'].transform(rolling_operation)
data['rolling_avg_speed'] = data['rolling_avg_speed'].fillna(data['speed'])

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

check = data.head(100000)
# Assuming 'data' is your DataFrame and includes all the necessary columns
# Preprocessing: Select relevant features and normalize
features = check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2', 'rolling_avg_speed']]
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Instantiate and fit the Isolation Forest model
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=42)
model.fit(features_normalized)

# Predict anomalies
check['anomaly'] = model.predict(features_normalized)
check['anomaly'].value_counts()

C:\Users\Hareem Raza\AppData\Local\Temp\ipykernel_10320\1559081916.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['anomaly'] = model.predict(features_normalized)


anomaly
 1    7579
-1    2421
Name: count, dtype: int64

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Assuming 'data' is your DataFrame and 'check' is a subset of the data
# Preprocessing: Select relevant features and normalize
features =  check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2', 'rolling_avg_speed']]
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Instantiate and fit the DBSCAN model
# Note: You need to choose appropriate values for eps and min_samples
dbscan = DBSCAN(eps=0.2, min_samples=10)
clusters = dbscan.fit_predict(features_normalized)

# Mark anomalies (DBSCAN labels outliers as -1)
check['anomaly_dbscan'] = (clusters == -1).astype(int)
# Review the results
print(check['anomaly_dbscan'].value_counts())

anomaly_dbscan
0    6226
1    3774
Name: count, dtype: int64


C:\Users\Hareem Raza\AppData\Local\Temp\ipykernel_10320\1958940382.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['anomaly_dbscan'] = (clusters == -1).astype(int)


In [ ]:
# Set the timestamp as the index
check.set_index('timestamps_Belgian', inplace=True)
# Sort the data by the index (timestamp)
check.sort_index(inplace=True)
# Select the feature for ARIMA modeling, e.g., 'speed'
time_series = check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'rolling_avg_speed']]

## VARIMA

In [ ]:
from sklearn.preprocessing import StandardScaler

features =  check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2', 'rolling_avg_speed']]
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

In [ ]:
from statsmodels.tsa.api import VARMAX
# Convert normalized features to DataFrame
features_df = pd.DataFrame(features_normalized, columns=features.columns, index=check.index)
# Fit VARIMA model
# Note: Determine the order (p, d, q) appropriately; here, we use (1,1,1) as an example
model = VARMAX(features_df, order=(1, 1), trend='c').fit(disp=False)

C:\Python310\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Python310\lib\site-packages\statsmodels\tsa\statespace\varmax.py:326: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  endog = np.require(endog.fillna(method='backfill'), requirements="W")


In [ ]:
# Get residuals
residuals = model.resid

# Anomaly detection - mark as anomaly if residuals for any variable are beyond a threshold
anomaly_threshold = 2.5  # Define based on your understanding of the data
check['anomaly'] = residuals.apply(lambda x: np.any(np.abs(x) > anomaly_threshold), axis=1).astype(int)

In [ ]:
# Check anomaly counts
print(check['anomaly'].value_counts())
# Optionally, inspect some of the anomalies
print(check[check['anomaly'] == 1])